In [1]:
from pytorch_lightning.utilities.model_summary import summarize
from datasets import load_dataset
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.common import ListDataset
from estimator import XformerEstimator
from gluonts.dataset.util import to_pandas
from gluonts.dataset.repository.datasets import get_dataset
from pytorch_lightning.loggers import CSVLogger

/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "
Need to compile C++ extensions to get sparse attention support. Please run python setup.py build develop
Triton is not available, some optimizations will not be enabled.
This is just a warning: No module named 'triton'
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Triton is not available, FusedMLP will not be enabled.


Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


In [2]:
#Tuning GluonTS models with Optuna
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import json
import optuna
import torch
from gluonts.mx import Trainer
from gluonts.evaluation import Evaluator

In [3]:
logger = CSVLogger("logs", name="vanilla")

In [4]:
freq = "1H"
prediction_length = 24

In [ ]:
# dataset = load_dataset("ett", "h2", prediction_length=24)
# freq = "1H"
# prediction_length = 24

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/240 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/120 [00:00<?, ? examples/s]

Dataset ett downloaded and prepared to /root/.cache/huggingface/datasets/ett/h2-prediction_length=24/1.0.0/4991b07785ef76f899777e38361fef7d338d97e7d0105656d2dadb3ca1572528. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
i = "solar-energy"# "electricity","traffic", "m4_hourly", "m4_daily", "m4_weekly", "m4_monthly", "m4_quarterly", "solar-energy"
dataset = get_dataset(i)
prediction_length = 24
freq = dataset.metadata.freq

In [5]:
int(dataset.metadata.feat_static_cat[0].cardinality)

137

In [ ]:
train_ds = ListDataset(dataset.train, freq=freq)
# val_ds = ListDataset(dataset.validation, freq=freq)
test_ds = ListDataset(dataset.test, freq=freq)

In [ ]:
## vanilla tranformer

In [6]:
class TransformerTuningObjective:  
    def __init__(self, dataset, prediction_length, freq, metric_type="mean_wQuantileLoss"):
        self.dataset = dataset
        self.prediction_length = prediction_length
        self.freq = freq
        self.metric_type = metric_type
    
    def get_params(self, trial) -> dict:
        return {
        "context_length": trial.suggest_int("context_length", dataset.metadata.prediction_length, dataset.metadata.prediction_length*7,4),
        "max_epochs": trial.suggest_int("max_epochs", 1, 10,2),
        "batch_size": trial.suggest_int("batch_size", 128, 256, 64),
        "num_encoder_layers": trial.suggest_int("num_encoder_layers", 2, 16,4),
        "num_decoder_layers": trial.suggest_int("num_decoder_layers", 2, 16,4),
        "hidden_layer_multiplier": trial.suggest_int("hidden_layer_multiplier", 1, 4, 1)
        }
     
    def __call__(self, trial):
        params = self.get_params(trial)
        estimator = XformerEstimator(
        freq=dataset.metadata.freq,
        prediction_length=dataset.metadata.prediction_length,
        context_length=params['context_length'],
        
        scaling=True,
        num_feat_static_cat=len(dataset.metadata.feat_static_cat),
        cardinality=[int(cat_feat_info.cardinality) for cat_feat_info in dataset.metadata.feat_static_cat],
        embedding_dimension=[5],
        
        nhead=2,
        num_encoder_layers=params['num_encoder_layers'],
        num_decoder_layers=params['num_decoder_layers'],
        hidden_layer_multiplier=params['hidden_layer_multiplier'],
        activation="gelu",
        # attention_args={"name": "global",},#global, nystrom
#         # longformer
#         attention_args={"name": "global",},
#         reversible=True, 
        
        # favor/performer
        attention_args={"name": "linformer", "iter_before_redraw": 2},
        
        batch_size=params['batch_size'],
        num_batches_per_epoch=100,
        trainer_kwargs=dict(max_epochs=params['max_epochs'], accelerator='auto', gpus=1, logger=logger),
    )
        predictor = estimator.train(
        training_data=self.dataset.train,
        # validation_data=val_ds,
        num_workers=8,
        # shuffle_buffer_length=1024
        )
        
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=self.dataset.test, 
            predictor=predictor
        )
        forecasts = list(forecast_it)
        # if layer == layers[0]:
        tss = list(ts_it)
        
        evaluator = Evaluator()
        agg_metrics, _ = evaluator(iter(tss), iter(forecasts))
        return agg_metrics[self.metric_type]

In [7]:
import time
start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(TransformerTuningObjective(dataset, prediction_length = prediction_length, freq=freq), n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2022-12-14 10:40:45,254] A new study created in memory with name: no-name-f5d8baeb-167a-475b-b368-e08d59964c7d
/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by [1, 10] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 9].
  low=low, old_high=old_high, high=high, step=step
/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/optuna/distributions.py:686: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  low=low, old_high=old_high, high=high, step=step
/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an inst

Epoch 0: : 0it [00:00, ?it/s]

/autofs/nccs-svm1_home1/hstellar/pytorch-transformer-ts/xformers/xformers/xformers/components/positional_embedding/sine.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  div = torch.exp(-math.log(10000) * (2 * (dim // 2) / self.dim_model))


Epoch 0: : 100it [00:11,  8.93it/s, loss=3.59, v_num=1, train_loss=4.190]

Epoch 0, global step 100: 'train_loss' reached 4.19059 (best 4.19059), saving model to 'logs/vanilla/version_1/checkpoints/epoch=0-step=100.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=1` reached.


...

Epoch 8: : 100it [00:07, 13.11it/s, loss=5.34, v_num=1, train_loss=5.350]

Epoch 8, global step 900: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=9` reached.


Epoch 8: : 100it [00:14,  6.71it/s, loss=5.34, v_num=1, train_loss=5.350]


[W 2022-12-14 10:47:14,854] Trial 2 failed because of the following error: ValueError("Expected parameter scale (Tensor of shape (128, 1)) of distribution StudentT(df: torch.Size([128, 1]), loc: torch.Size([128, 1]), scale: torch.Size([128, 1])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:\ntensor([[1.2161e-06],\n        [2.1450e-16],\n        [9.7759e-01],\n        [8.2926e-09],\n        [5.8307e-07],\n        [4.0082e-12],\n        [1.9817e-12],\n        [1.6628e-06],\n        [5.1138e+00],\n        [1.0228e-29],\n        [2.9117e-33],\n        [1.0496e-42],\n        [2.3228e-13],\n        [0.0000e+00],\n        [3.7915e-18],\n        [6.0728e-35],\n        [1.1549e-14],\n        [6.0975e-10],\n        [2.0302e-16],\n        [2.2327e-11],\n        [5.6052e-45],\n        [1.4711e-08],\n        [2.4608e-17],\n        [5.3401e-05],\n        [0.0000e+00],\n        [8.4417e-14],\n        [9.8802e-10],\n        [6.7562e+00],\n        [1.3582e-08],\n    

ValueError: Expected parameter scale (Tensor of shape (128, 1)) of distribution StudentT(df: torch.Size([128, 1]), loc: torch.Size([128, 1]), scale: torch.Size([128, 1])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[1.2161e-06],
        [2.1450e-16],
        [9.7759e-01],
        [8.2926e-09],
        [5.8307e-07],
        [4.0082e-12],
        [1.9817e-12],
        [1.6628e-06],
        [5.1138e+00],
        [1.0228e-29],
        [2.9117e-33],
        [1.0496e-42],
        [2.3228e-13],
        [0.0000e+00],
        [3.7915e-18],
        [6.0728e-35],
        [1.1549e-14],
        [6.0975e-10],
        [2.0302e-16],
        [2.2327e-11],
        [5.6052e-45],
        [1.4711e-08],
        [2.4608e-17],
        [5.3401e-05],
        [0.0000e+00],
        [8.4417e-14],
        [9.8802e-10],
        [6.7562e+00],
        [1.3582e-08],
        [2.2228e-17],
        [3.6318e-07],
        [4.6966e-08],
        [1.2684e-23],
        [4.8149e-18],